In [7]:
println("Start!!!")
// spark 初始化，为后续sql的运行准备环境
import $ivy.`com.alibaba:fastjson:1.2.70`
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}
var user = sys.env.get("JUPYTERHUB_USER").get
val conf = {
   new SparkConf()
   .setAppName(s"Online-Feature-For-Bid-${user}")
   .setMaster("yarn")
   .set("spark.yarn.queue", "root.zw03.hadoop-waimaiadrd.etl") // spark 的队列，可以根据自己的需要设置
   .set("spark.yarn.archive", "viewfs://hadoop-meituan/user/hadoop-hdp/notebook/spark-jars/spark_libs.zip")
   .set("spark.yarn.stagingDir", "viewfs://hadoop-meituan/ghnn03/sparkstaging")
   .set("spark.executor.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.driver.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.executorEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.yarn.appMasterEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.sql.hive.metastorePartitionPruning", "true")
   .set("spark.shuffle.service.enabled", "true")
   .set("spark.dynamicAllocation.enabled", "true")
   .set("spark.dynamicAllocation.minExecutors", "600") //spark的执行Executors
   .set("spark.dynamicAllocation.maxExecutors","1500")
   .set("spark.executor.memory","16G")
   .set("spark.driver.memory","7G")
   .set("spark.input.dir.recursive","true")
   .set("spark.sql.ignore.existed.function.enable","true")
   .set("spark.yarn.maxAppAttempts", "2")
   .set("spark.noResourceMaxInterval","86400s")
   .set("spark.scheduler.mode", "FAIR")
   .set("spark.executor.cores","8")
   .set("spark.yarn.executor.memoryOverhead","32024")
   .set("spark.yarn.driver.memoryOverhead","32024")
   .set("spark.hadoop.hive.mt.renew.token.enable", "true")
}
val spark = {
 NotebookSparkSession
   .builder()
   .config(conf)
   .enableHiveSupport.getOrCreate
}
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
spark.sparkContext.applicationId
var logo="AFO_RL"
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
import java.text.SimpleDateFormat
import java.util.Date

// 通用udf注册
import scala.collection.mutable.ArrayBuffer
import scala.math
import scala.util.Random;
import java.lang.Math
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}

import scala.collection.immutable.Map
import scala.collection.mutable

def JSONObj2Map(jsonObj: JSONObject): Map[String, String] = {
    var map = mutable.Map[String, String]()
    if(!jsonObj.isEmpty){
        val itr: java.util.Iterator[String] = jsonObj.keySet().iterator()
        while (itr.hasNext) {
            val key = itr.next()
            map += ((key, jsonObj.getString(key)))
        }
    }
    map.toMap
}

import org.apache.hadoop.hive.ql.exec.UDF
import spark.implicits._
val findIndex2 =  (strQuote:String) => {
    val poi = strQuote.split("\073").last
    val ind = strQuote.split("\073").zipWithIndex.filter(_._1 == poi).map(_._2).min
    ind.toString
}
spark.sqlContext.udf.register("findIndex2", findIndex2)
spark.udf.register("findIndex2", findIndex2)

def getDate(delta:Int, current_day:String = "today"):String = {
    val sdf:SimpleDateFormat = new SimpleDateFormat("yyyyMMdd")
    val cal:Calendar = Calendar.getInstance()
    if ( !current_day.equals("today")){
      val dt = sdf.parse(current_day)
      cal.setTime(dt)
    }
    cal.add(Calendar.DATE,delta)
    val day = sdf.format(cal.getTime)
    day
}
def getTimeSuffix():String={
    val dateFormat = new SimpleDateFormat("yyyyMMdd")
    val cla = Calendar.getInstance()
    cla.setTimeInMillis(System.currentTimeMillis())
    val table_suffix = dateFormat.format(cla.getTime)
    table_suffix
}
def dropTable(sqlContext:SQLContext, tableName:String, is_execute_sql:Boolean):Unit={
    printCurTime()
    var sql = s"Drop table if exists ${tableName}"
    if(is_execute_sql){
        println(s"执行 \n ${sql}")
        sqlContext.sql(sql)
    }
    printCurTime()
}

def executeSQL(sqlContext:SQLContext, sql:String, is_execute_sql:Boolean):DataFrame={
    printCurTime()
    println(s"执行 \n ${sql}")
    var result_ = sqlContext.sql(sql)
    printCurTime()
    result_
}

def genDateList(start: String, end: String): List[String] = {
    import java.util.Date
    import org.joda.time.{DateTime, Instant, Days}
    import org.joda.time.format.{DateTimeFormat, DateTimeFormatter}

    val DATE_KEY_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")
    val DATE_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")

    val startDate = DATE_KEY_FORMAT.parseDateTime(start).toDateTime
    val endDate = DATE_KEY_FORMAT.parseDateTime(end).toDateTime

    val daysCount = Days.daysBetween(startDate, endDate).getDays() + 1
    (0 until daysCount).map(startDate.plusDays(_)).map(d => d.toString(DATE_FORMAT)).toList
  }



def printCurTime():Unit={
    println(new SimpleDateFormat("yyyy-MM-dd HH:mm:ss").format(new Date))
}


def showTopNFromSQL(sql:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,sql,true)
    result.show(n)
    printCurTime()
}
def executeSQLLazy(sql:String):DataFrame={
    executeSQL(spark.sqlContext,sql,true)
}
def showTopNFromTableName(tableName:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,s"""select * from ${tableName} limit ${n}""",true)
    result.show(n)
    printCurTime()
}
def executeShell(shell:String):Unit={
    printCurTime()
    println("----------------The content of shell is: -------------------")
    println(shell)
    println("----------------Start execution....-------------------")
    var file_name=System.currentTimeMillis()
    val writer = new PrintWriter(new File(s"/opt/meituan/dolphinfs_${user}/${file_name}.sh" ))
    writer.write(shell)
    writer.close()
    //启动脚本
    s"chmod +777 /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"sh /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"rm -r /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    println("----------------End-------------------")
    printCurTime()
}

def getFeatureMap(x:((String,String))):String={
    var x_1 = x._1
    var x_2 = x._2.replace("Type","")
    if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
    }
    var x_3 = s"""${x_1} ${x_2}"""
    x_3
}
// 传入sql，返回结果表
def createTableFromSQLWithName(sql:String,tableName:String,database:String):Unit={
    var rowData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var finalTableName=database+"."+tableName
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    
    
    //var featureForCreateTable=table_scheme.map{x=> s"""${x._1} ${x._2.stripSuffix("Type")}""".replace("Type","")}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    executeSQLLazy(s"insert OVERWRITE table ${finalTableName} select ${columnsName} from ${tmpTableName}")
}

def createTableFromSQL(sql:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,"tmp")
    "tmp."+finalTableName
}
def createTableFromSQL(sql:String,database:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,database)
    database+"."+finalTableName
}

def generateTrajectory(tableName:String, groupColumns:Array[String], rankColumns:Array[String], joinColumns:Array[String],database:String):String={
    
    println(tableName.split("."))
    var finalTableName=database+"."+tableName.split("\\.")(1)+"_trajectory"
    
    var table_scheme = executeSQLLazy(s"""select * from ${tableName} limit 1""").dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var groupColumnString=groupColumns.mkString(",")
    var rankColumnString = rankColumns.mkString(",")
    var joinConditionString = joinColumns.map(x=> s"t1.${x}=t2.${x} ").mkString("\n and ")
    var selectT1columnsNameString = table_scheme.map{x=> s"t1.${x._1}"}.mkString(",")
    var selectT2columnsNameString = table_scheme.map{x=> s"t2.${x._1} as next_${x._1}"}.mkString(",")
    //var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",") + ",rn int,"+table_scheme.map(x=> {
        var x_1 = x._1
        var x_2 = x._2.replace("Type","")
        if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
        }
        
        s"""next_${x_1} ${x_2}"""}
                                              ).mkString(",")+",next_rn int"
    var allColumnString=table_scheme.map{x=> s"${x._1}"}.mkString(",")+",rn,"+table_scheme.map{x=> s"next_${x._1}"}.mkString(",")+",next_rn"
    var sql=s"""
        select ${selectT1columnsNameString},t1.rn,${selectT2columnsNameString},t2.rn as next_rn
        from(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t1
        left join(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t2
        on t1.rn+1 = t2.rn
        and ${joinConditionString}
        """
    var trajectoryData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    trajectoryData.createOrReplaceTempView(tmpTableName)
    spark.sql(s"insert OVERWRITE table ${finalTableName} select ${allColumnString} from ${tmpTableName}")
    finalTableName
}
def createPTTableFromTable(tableName:String,targetTableName:String,database:String):String={
    var finalTableName=targetTableName
    var rowData=executeSQLLazy(s"select * from ${tableName} limit 1")
    var tmpTableName=s"${database}.${logo}_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt='${date_s}') select ${columnsName} from ${tmpTableName}")
    finalTableName
}

def createPTTableFromTableWithSpace(tableName:String,database:String):String={
    var finalTableName=database+s".${logo}_${System.currentTimeMillis()}"
    var rowData=executeSQLLazy(s"select * from ${tableName}")
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt=${date_s}) select ${columnsName} from ${tmpTableName}")
    finalTableName
}

Start!!!
Creating SparkSession


Spark UI

import $ivy.$                            

import com.alibaba.fastjson.{JSON, JSONObject}


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.{SparkConf, SparkContext}

user: String = "wanghao251"
conf: SparkConf = org.apache.spark.SparkConf@5c5cf0a1
spark: SparkSession = org.apache.spark.sql.SparkSession@11be1a14
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

res6_16: String = "application_1671247042382_1846793"
logo: String = "AFO_RL"
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

import java.text.SimpleDateFormat

import java.util.Date

// 通用udf注册

import scala.collection.mutable.ArrayBuffer

import scala.math

import scala.util.Random;

import java.lang.Math

import com.alibaba.fastjson.{JSON, JSONObject}


import or

In [13]:

var train_table_name ="mart_waimaiad.AFO_RL_1674030629973"
var predict_table_name="mart_waimaiad.AFO_RL_1674030629973"

var reward="reward"//RL reward 字段名字
var poi_id="poi_id"// pv 后续会去掉，修改为key
var pvid="pvid"// pv 后续会去掉，修改为key
var action="action"//RL action 字段名字
var is_terminal="is_terminal" //RL terminal 字段
var path_prefix=s"viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/${user}" //模型和tf recoder位置


var start_date="20230101" 
var end_date="20230118" 

var trainDt=s"""pt=20230117"""
var predictDt=s"""pt=20230117"""

var model_name="R-BCQ" //是否使用REM
var remark="1229"

var task_type="high"
var use_bcorle="False"
var use_adaptive="False"
var use_batch_loss="True"
var use_mask="False"
var lambda_alpha="1e-1_1e-1_1e-1_1e-1"
var lambda_update_interval="100"
var lambda_budgets_target= "1_1_1_1"
var constraint_target="1_1_1_1"
var constraint_loss_weight="0.01_0.01_0.01_0.01"

var high_model_dir="viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/20221228/REM_1228_high/high"
var high_mean_var_dir="dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1672289508416/mtjupter/OS/wanghao/allinone/data/high_mean_var_of_state.20221227"

var high_state_dynamic_fea="ctr,histctr,aimcpc,impr,cpc,revenue,click,order_num,mt_butie,impr_before1week,click_before1week,cpc_before1week,order_num_before1week,revenue_before1week,mt_butie_before1week,charge_before1week,impr_before2week,click_before2week,cpc_before2week,revenue_before2week,charge_before2week,order_num_before2week,mt_butie_before2week,impr_before3week,cpc_before3week,click_before3week,order_num_before3week,mt_butie_before3week,revenue_before3week,charge_before3week,impr_before4week,charge_before4week,revenue_before4week,mt_butie_before4week,cpc_before4week,click_before4week,order_num_before4week".toLowerCase()
var low_state_dynamic_fea="ctr,histctr,aimcpc,impr,cpc,revenue,click,order_num,mt_butie,impr_before1week,click_before1week,cpc_before1week,order_num_before1week,revenue_before1week,mt_butie_before1week,charge_before1week,impr_before2week,click_before2week,cpc_before2week,revenue_before2week,charge_before2week,order_num_before2week,mt_butie_before2week,impr_before3week,cpc_before3week,click_before3week,order_num_before3week,mt_butie_before3week,revenue_before3week,charge_before3week,impr_before4week,charge_before4week,revenue_before4week,mt_butie_before4week,cpc_before4week,click_before4week,order_num_before4week,user_tag_pref_click_3day,user_ord_num_weekdays,feacomp_cvr,lowerk,real_cpc,real_csm,winningscore_comp12,real_cst_bdgt,real_aBln,feacomp_ctr,real_expose,real_revenue,ratioctr,real_price,lowerbid,ratio_aimcpc,real_budget,real_csm_bln,real_click,winningscore_qs,ratio_click,ratioctr_exp,upperk,charge,real_order,ratio_cpc,real_bln".toLowerCase()
var high_state_cate_fea="poi_id"
var low_state_cate_fea="poi_id,cityid,hour"

var parameter_servers=1 //ps的数量
var workers=2 //work的数量
var batch_size="256" // batch size
var epoch="10"

var account=user //AFO框架启动消息通知对象
var queue="root.zw05_training_cluster.hadoop-waimaiadrd.cpu_job"
// 模型配置区

var learning_rate="0.00001"
var deep_layers="128,64,64" //网络设置
var gamma="1.0"
var num_action="20" //action的个数
var update_interval="300"
var embed_dim="8" //离散特征的embedding维度
var bit="22"//离散特征的空间
var ext_is_predict_serving="0"
var use_bn="False"
var i_loss_weight="0" //使用BCQ时iloss的权重
var i_regularization_weight="0" 
var q_loss_weight="1" //使用BCQ时qloss的权重
var num_heads="20" //REM head 个数
var threshold="0.3" // BCQ阈值
var use_cate_feature="True"
var use_dense_hashtable="True" //是否使用dense_hashtable
var model_file="udm/tangbo17/rbcq.py" //用户自定义模型代码文件
var readFromHive="True"
var file_path="allinone" // 模型的文件目录
var mod="local" //mod=local 代表启动workbench本地调试，其他代表提交到服务器
var code_branch = "master"

train_table_name: String = "mart_waimaiad.AFO_RL_1674030629973"
predict_table_name: String = "mart_waimaiad.AFO_RL_1674030629973"
reward: String = "reward"
poi_id: String = "poi_id"
pvid: String = "pvid"
action: String = "action"
is_terminal: String = "is_terminal"
path_prefix: String = "viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251"
start_date: String = "20230101"
end_date: String = "20230118"
trainDt: String = "pt=20230117"
predictDt: String = "pt=20230117"
model_name: String = "REM"
remark: String = "1229"
task_type: String = "high"
use_bcorle: String = "False"
use_adaptive: String = "False"
use_batch_loss: String = "True"
use_mask: String = "False"
lambda_alpha: String = "1e-1_1e-1_1e-1_1e-1"
lambda_update_interval: String = "100"
lambda_budgets_target: String = "1_1_1_1"
constraint_target: String = "1_1_1_1"
constraint_loss_weight: String = "0.01_0.01_0.01_0.01"
high_model_dir: String = "viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/20221228/R

In [14]:

var task="train" 
var with_chief="true"
var remark="high_0101_0117_epoch10"

mod="local1" 
var train_cmd=s"""
    if [[ ! -d "/opt/meituan/dolphinfs_${user}/${train_table_name}/mtjupter" ]]; 
    then
        echo "文件夹不存在"
        mkdir /opt/meituan/dolphinfs_${user}/${train_table_name}
        cd /opt/meituan/dolphinfs_${user}/${train_table_name}
        git clone ssh://git@git.sankuai.com/~tangbo17/mtjupter.git
        git checkout ${code_branch}
    else
        echo "文件夹存在"
    fi

    cd /opt/meituan/dolphinfs_${user}/${train_table_name}/mtjupter
    git pull
    ls
    cd OS/wanghao/${file_path}
    echo "开始train model"
    cp ${high_mean_var_dir} ./data/
    sh run.sh ${train_table_name} ${predict_table_name} ${model_name} ${batch_size} ${epoch} ${learning_rate} ${deep_layers} ${gamma} ${num_action} ${update_interval} ${embed_dim} ${bit} ${ext_is_predict_serving} ${use_bn} ${i_loss_weight} ${i_regularization_weight} ${q_loss_weight} ${num_heads} ${threshold} ${path_prefix} ${start_date} ${end_date} ${high_state_dynamic_fea} ${low_state_dynamic_fea} ${task} ${parameter_servers} ${workers} ${account} ${queue} ${with_chief} ${use_dense_hashtable} ${high_state_cate_fea} ${low_state_cate_fea} ${use_cate_feature} ${readFromHive} ${file_path} ${mod} ${train_table_name}/${trainDt} ${use_adaptive} ${use_batch_loss} ${lambda_alpha} ${lambda_update_interval} ${lambda_budgets_target} ${constraint_target} ${constraint_loss_weight} ${use_mask} ${use_bcorle} ${task_type} ${high_model_dir} ${remark} ${high_mean_var_dir}
    """
executeShell(train_cmd)


2023-01-18 17:27:21
----------------The content of shell is: -------------------

    if [[ ! -d "/opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1674030629973/mtjupter" ]]; 
    then
        echo "文件夹不存在"
        mkdir /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1674030629973
        cd /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1674030629973
        git clone ssh://git@git.sankuai.com/~tangbo17/mtjupter.git
        git checkout master
    else
        echo "文件夹存在"
    fi

    cd /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1674030629973/mtjupter
    git pull
    ls
    cd OS/wanghao/allinone
    echo "开始train model"
    cp dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1672289508416/mtjupter/OS/wanghao/allinone/data/high_mean_var_of_state.20221227 ./data/
    sh run.sh mart_waimaiad.AFO_RL_1674030629973 mart_waimaiad.AFO_RL_1674030629973 REM 256 10 0.00001 128,64,64 1.0 20 300 8 22 0 False 0 0 1 20 0.3 viewfs://hadoop-meituan/user/hadoop-hmart-waima

美团代码仓库(DevTools Code)                                 代码仓库组
[连接信息]:         zf-proxy01 => f4de250a-72d9-4718-ad3a-2258c4722020
[路由信息]:                                 slave-mcode_shard_03-gh-01
🏮🏮━︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵━🏮🏮
 / **･゜ﾟ･*:.｡..｡.:*･･*:.｡. .｡.:*･゜ﾟ･･゜ﾟ･*:.｡..｡.:*･･*:.｡. .｡.:*･゜\
  |        辞旧迎新，DevTools Code团队祝童鞋们2023兔年快乐!         |
 \ *･゜ﾟ･*:.｡..｡.:*･･*:.｡. .｡.:*･゜ﾟ･*･゜ﾟ･*:.｡..｡.:*･･*:.｡. .｡.:*･｡./
  -------------------------------------------------------------------
                   ❄️        🎉阖家欢乐🎉        ❄️              
          卯         ❄️                        ❄️            兔 
          門        ❄️                          ❄️           歲 
          生     ❄️                                ❄️        報 
          喜       ❄️   Happy Spring Festival!   ❄️          新 
          氣         ❄️                        ❄️            春
               >>>>>> Better Decade Better Code <<<<<<


Already up-to-date.
common
OS
personal
PT
readme.md
RecCpc
test.py
tool
开始train model


cp: cannot stat ‘dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1672289508416/mtjupter/OS/wanghao/allinone/data/high_mean_var_of_state.20221227’: No such file or directory
+ set -u
+ set -o pipefail
+ table_name=mart_waimaiad.AFO_RL_1674030629973
+ predict_table_name=mart_waimaiad.AFO_RL_1674030629973
+ algorithm_name=REM
+ batch_size=256
+ epoch=10
+ learning_rate=0.00001
+ deep_layers=128,64,64
+ gamma=1.0
+ num_action=20
+ update_interval=300
+ embed_dim=8
+ bit=22
+ ext_is_predict_serving=0
+ use_bn=False
+ i_loss_weight=0
+ i_regularization_weight=0
+ q_loss_weight=1
+ num_heads=20
+ threshold=0.3
+ path_prefix=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251
+ start_date=20230101
+ end_date=20230118
+ high_state_dynamic_fea=ctr,histctr,aimcpc,impr,cpc,revenue,click,order_num,mt_butie,impr_before1week,click_before1week,cpc_before1week,order_num_before1week,revenue_before1week,mt_butie_before1week,charge_before1week,impr_before2week,click_before2week,cpc_before2week,revenue

use_bcq:False
use_rem:True


...

+ echo predict_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_tfrecoder/20230118
+ echo ext_is_predict_serving:0


ext_is_predict_serving:0


+ case ${mod} in
+ hope run --xml conf/dist.xml --usergroup hadoop-hmart-waimaiad -Dworker.memory=102400 -Dafo.engine.wait_for_job_finished=true -Ddistribute.mode=ps '-Dworker.script=python rl_code/main.py' '-Dparameter.server.script=python rl_code/main.py' '-Dchief.script=python rl_code/main.py' -Dboard.log_dir=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/20230118/REM_high_0101_0117_epoch10/high -Dparameter.servers=1 -Dargs.ext_is_predict_serving=0 -Dworkers=2 -Dwith.chief=true -Dafo.xm.notice.receivers.account=wanghao251 -Dafo.app.name=REM_high_0101_0117_epoch10 -Dqueue=root.zw05_training_cluster.hadoop-waimaiadrd.cpu_job -Dargs.log_dir=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/20230118/REM_high_0101_0117_epoch10/high -Dargs.high_state_dynamic_fea_mean_var_filename=data/high_mean_var_of_state.20230118 -Dargs.low_state_dynamic_fea_mean_var_filename=data/low_mean_var_of_state.20230118 -Dargs.task=train -Dargs.is_dist=1 -Dargs.batch_size=

2023-01-18 17:27:24 [INFO] Staged file or directory: /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1674030629973/mtjupter/OS/wanghao/allinone/conf/dist.xml
2023-01-18 17:27:24 [INFO] Job info initialized at /workdir/sankuai/.hope/template_jobinfo/4c896d18de02f0c5c5aebab3c8846dbd
2023-01-18 17:27:24 [INFO] Staged file or directory: /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1674030629973/mtjupter/OS/wanghao/allinone
2023-01-18 17:27:25 [INFO] Hope config check success
2023-01-18 17:27:25 [INFO] File name: /workdir/sankuai/.hope/job-519e4474_9712_11ed_aa2d_161ba065b71f/dist.tar.gz
2023-01-18 17:27:25 [INFO] File size: 106889B
2023-01-18 17:27:25 [INFO] Uploading...
2023-01-18 17:27:25 [INFO] Upload success
**********************************
 [INFO] code version save success 
**********************************
*************************************
 job submit success, run_id: 1158299 
*************************************
download file: http://s3plus.vip.sankuai.com/v

task: String = "train"
with_chief: String = "true"
remark: String = "high_0101_0117_epoch10"
train_cmd: String = """
    if [[ ! -d "/opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1674030629973/mtjupter" ]]; 
    then
        echo "文件夹不存在"
        mkdir /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1674030629973
        cd /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1674030629973
        git clone ssh://git@git.sankuai.com/~tangbo17/mtjupter.git
        git checkout master
    else
        echo "文件夹存在"
    fi

    cd /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1674030629973/mtjupter
    git pull
    ls
    cd OS/wanghao/allinone
    echo "开始train model"
    cp dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1672289508416/mtjupter/OS/wanghao/allinone/data/high_mean_var_of_state.20221227 ./data/
    sh run.sh mart_waimaiad.AFO_RL_1674030629973 mart_waimaiad.AFO_RL_1674030629973 REM 256 10 0.00001 128,64,64 1.0 20 300 8 22 0 False 0 0 1 20 0.3 viewfs://had

In [6]:
showTopNFromSQL("""
select count(*) from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230113_high where real_bln=0
""",1000)

2023-01-14 15:32:34
2023-01-14 15:32:34
执行 
 
select count(*) from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230113_high where real_bln=0

2023-01-14 15:32:34


show at cmd0.sc:161

2000 / 2000

[Stage 4:====================================================>(1999 + 1) / 2000]

show at cmd0.sc:161

1 / 1

+--------+
|count(1)|
+--------+
|49228619|
+--------+

2023-01-14 15:32:37


In [10]:
showTopNFromSQL("""
select count(*) from mart_waimaiad.AFO_RL_1673613376579 where budget is null
""",10)

2023-01-14 15:41:29
2023-01-14 15:41:29
执行 
 
select count(*) from mart_waimaiad.AFO_RL_1673613376579 where budget is null

2023-01-14 15:41:29


show at cmd0.sc:161

200 / 200

[Stage 8:======================================================>(199 + 1) / 200]

show at cmd0.sc:161

1 / 1

+--------+
|count(1)|
+--------+
|       0|
+--------+

2023-01-14 15:41:39


In [5]:
showTopNFromSQL("""
select count(*) from mart_waimaiad.AFO_RL_1673758698266 where real_aBln[0]+real_aBln[1]+real_aBln[2]+real_aBln[3]=0
""",10)

2023-01-16 19:32:38
2023-01-16 19:32:38
执行 
 
select count(*) from mart_waimaiad.AFO_RL_1673758698266 where real_aBln[0]+real_aBln[1]+real_aBln[2]+real_aBln[3]=0

2023-01-16 19:32:38


show at cmd0.sc:161

200 / 200

[Stage 4:======================================================>(199 + 1) / 200]

show at cmd0.sc:161

1 / 1

+---------+
| count(1)|
+---------+
|433662366|
+---------+

2023-01-16 19:33:10


In [4]:
showTopNFromSQL("""
select count(*) from mart_waimaiad.AFO_RL_1673758698266 where real_bln[0]+real_bln[1]+real_bln[2]+real_bln[3]!=0
""",10)

2023-01-16 19:25:35
2023-01-16 19:25:35
执行 
 
select count(*) from mart_waimaiad.AFO_RL_1673758698266 where real_bln[0]+real_bln[1]+real_bln[2]+real_bln[3]!=0

2023-01-16 19:25:35


show at cmd0.sc:161

200 / 200

[Stage 2:======================================================>(199 + 1) / 200]

show at cmd0.sc:161

1 / 1

+---------+
| count(1)|
+---------+
|132197494|
+---------+

2023-01-16 19:26:45
